<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/crawer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
from bs4 import BeautifulSoup
# 品書閣
url="https://tw.pinsuge.com/index/85767.html"
sites="https://tw.pinsuge.com"
req = requests.get(url)
soup=BeautifulSoup(req.text)
soup

<!DOCTYPE html>
<html><head><meta charset="utf-8"/><title>虧成世界首富了小說_君無雪_虧成世界首富了筆趣閣最新章節列表在線全文免費閱讀_吧_txt_123_5200_sodu_KUICHENGSHIJIESHOUFULE__品書閣手機版</title><meta content="虧成世界首富了,君無雪,虧成世界首富了最新章節,虧成世界首富了全文閱讀,虧成世界首富了章節列表,虧成世界首富了txt,虧成世界首富了全文免費閱讀,虧成世界首富了在線閱讀,虧成世界首富了目錄,虧成世界首富了無彈窗,KUICHENGSHIJIESHOUFULE,KUI成世界首富了,,虧成世界首富了吧,txt,頂點,起點,筆趣閣,123,百度貼吧,5200,sodu,網易云閱讀" name="keywords"/><meta content="品書閣免費提供虧成世界首富了最新章節列表目錄免費閱讀。" name="description"/><meta content="width=device-width,initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="320" name="MobileOptimized"/><meta content="portrait" name="screen-orientation"/><meta content="portrait" name="x5-orientation"/><meta content="yes" name="apple-mobile-web-app-capable"/><link href="/inc/common.css" rel="stylesheet"/><script async="" data-ad-client="ca-pub-7256405247152220" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script></head><body><

In [ ]:
articles=soup.find_all(name="a")
links=[]

for itm in articles[1:]:
  if not itm.get("title"):
    continue 
  links.append([itm.get("title",),"{}{}".format(sites,itm.get("href"))])

links

In [ ]:
index=1
for link in links:
  reg=requests.get(link[1])
  soup=BeautifulSoup(reg.text)
  content=soup.find(name="div",id="content").find_all(name="p")
  adv_text="喜歡虧成世界首富了請大家收藏"
  to_text=''
  to_text+="%s \n" %link[0]
  for i,itm in enumerate(content[1:-1]):
    txt=itm.getText()
    if i==1:
      txt=txt[:txt.rfind("。")+1]
    if adv_text in txt:
      continue
    to_text+="%s \n" %txt
  with open("%03d.txt" % index,mode="w",encoding="utf-8") as f:
    f.write(to_text)
  index+=1


In [ ]:
files=os.listdir()
files=[file for file in files if file.endswith(".txt")]
files.sort()
files
for file in files:
  os.system("cat {}>> ouptput.txt".format(file))